### Conda Env

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:48
🔁 Restarting kernel...


In [ ]:
!conda --version

!conda install geopandas
!conda install rioxarray
!conda install geojson
!conda install rasterstats

conda 4.9.2
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [ ]:
# https://stackoverflow.com/questions/49154899/resolvepackagenotfound-create-env-using-conda-and-yml-file-on-macos

#!pip install rioxarray
#!pip install xarray-spatial
!conda install cffi==1.15.0

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cffi==1.15.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cffi-1.15.0                |   py37h036bc23_0         225 KB  conda-forge
 

In [ ]:
!conda list

# packages in environment at /usr/local:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
affine                    2.3.0                    pypi_0    pypi
attrs                     21.4.0             pyhd8ed1ab_0    conda-forge
bokeh                     2.4.2            py37h89c1867_0    conda-forge
boost-cpp                 1.74.0               hc6e9bd1_3    conda-forge
branca                    0.4.2              pyhd8ed1ab_0    conda-forge
brotlipy                  0.7.0           py37h5e8e339_1001    conda-forge
bzip2                     1.0.8                h7f98852_4    conda-forge
c-ares                    1.17.1               h7f98852_1    conda-forge
ca-certificates           2021.10.8            ha878542_0    conda-forge
cairo                     1.16.0            h6cf1ce9_1008    conda-forge
certifi            

## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install geopandas
!pip install rioxarray
!pip install geojson
!pip install rasterstats

import os
from glob import glob
import numpy as np
import pandas as pd
import geojson
import shapely as shp
from shapely.geometry import Polygon
import geopandas as gpd
import xarray as xr
import rioxarray as rioxr

## Load All Parcels

In [3]:
# Load all parcels
os.chdir('/content/drive/MyDrive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Coastline_parcels_polygon')
path_parcels = glob(os.path.join(os.getcwd(), 'parcels.geojson'))[0]
parcels = gpd.read_file(path_parcels)
parcels.rename(columns={'id':'parcelID'}, inplace=True)
parcels.tail()

,GID_0,NAME_0,xcentroid,ycentroid,rep_val,parcelID,geometry
261,VTM,Vietnam,107.790080,16.061970,61.926025,261,"MULTIPOLYGON (((110.06040 16.71273, 107.04268 ..."
262,VTM,Vietnam,109.127039,14.925381,62.026210,262,"MULTIPOLYGON (((107.42597 13.68468, 107.04268 ..."
263,VTM,Vietnam,105.494704,18.896489,62.195597,263,"MULTIPOLYGON (((107.77233 18.59714, 104.76500 ..."
264,VTM,Vietnam,106.311819,20.578156,63.444988,264,"MULTIPOLYGON (((106.01894 22.31722, 107.99597 ..."
265,VTM,Vietnam,107.390897,21.171024,64.280961,265,"MULTIPOLYGON (((106.79812 22.76943, 106.72292 ..."


In [4]:
#parcels.to_file('parcels.geojson', driver='GeoJSON')
with open('parcels.geojson') as f:
  parcels_geojson = geojson.load(f)

## Tile Ponds by Parcels

In [13]:
# Load All Ponds
path_ponds = '/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/Asia_aquaculture_ponds_2019_plusGADM.gpkg'
ponds_all = gpd.read_file(path_ponds)
ponds_all.head()

,id,parcel_number,distance,elevation,GID_0,NAME_0,GID_1,NAME_1,ENGTYPE_1,GID_2,NAME_2,geometry
0,88082,5302,7845.0,20,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83395 21.60565, 107.83404 ..."
1,88086,5302,7814.0,17,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83440 21.60511, 107.83440 ..."
2,88088,5302,7326.0,21,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83467 21.60089, 107.83484 ..."
3,88089,5302,7837.0,13,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83484 21.60556, 107.83502 ..."
4,88095,5302,7764.0,18,VNM,Vietnam,VNM.49_1,Qu?ng Ninh,Province,VNM.49.10_1,Móng Cái,"MULTIPOLYGON (((107.83538 21.60466, 107.83538 ..."


In [74]:
os.chdir('/content/drive/My Drive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Aquaculture_ponds_Asian_coast_2019/ponds_by_parcel/')
os.getcwd()

# Build Generator Function to tile ponds
# Reference: https://stackoverflow.com/questions/37156574/why-does-a-generator-expression-need-a-lot-of-memory/37156765:
def tile_ponds_by_parcel(n):
  parcelID = 95
  while parcelID < n:
    ponds_sub = ponds_all[ponds_all.geometry.within(parcels[parcels['parcelID']==parcelID].geometry.unary_union)]
    if ponds_sub.empty:
      del ponds_sub
    else:
      yield ponds_sub.to_file('ponds_pa_'+str(parcelID)+'.geojson', driver='GeoJSON')
    parcelID += 1

gen = tile_ponds_by_parcel(len(parcels['parcelID']))

In [75]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

## Tidy rasters

In [5]:
# Pair up Index and Years
keys = [str(i) for i in range(36)]
#years = pd.date_range(start='1984', end='2020', freq='Y')
years = [i for i in range(1984,2020)]
pair = dict([[i,j] for i,j in zip(keys, years)])
print(pair)

{'0': 1984, '1': 1985, '2': 1986, '3': 1987, '4': 1988, '5': 1989, '6': 1990, '7': 1991, '8': 1992, '9': 1993, '10': 1994, '11': 1995, '12': 1996, '13': 1997, '14': 1998, '15': 1999, '16': 2000, '17': 2001, '18': 2002, '19': 2003, '20': 2004, '21': 2005, '22': 2006, '23': 2007, '24': 2008, '25': 2009, '26': 2010, '27': 2011, '28': 2012, '29': 2013, '30': 2014, '31': 2015, '32': 2016, '33': 2017, '34': 2018, '35': 2019}


In [26]:
path_img = '/content/drive/My Drive/myExportImage/parcels_msk_30m/original/'
items = glob(os.path.join(path_img, '*.tif'))
items

['/content/drive/My Drive/myExportImage/parcels_msk_30m/original/p_151_B.tif',
 '/content/drive/My Drive/myExportImage/parcels_msk_30m/original/p_151_A.tif',
 '/content/drive/My Drive/myExportImage/parcels_msk_30m/original/p_11_A.tif',
 '/content/drive/My Drive/myExportImage/parcels_msk_30m/original/p_11_B.tif']

In [52]:
elements = [rioxr.open_rasterio(i) for i in items]
xdas = [i.rename({'band':'time'}) for i in elements]
xdss = [i.to_dataset(name='watermask') for i in xdas]
xdss = [i.reindex(y = list(reversed(i.y))) for i in xdss]

parcelIDs = [i.split('/')[-1].split('_')[1] for i in items]
for i in range(len(parcelIDs)):
  xdss[i].attrs['parcelID'] = parcelIDs[i]
  
# How to write this in list comprehension? 
# For-loop is the proper way for such case, not list comprehension (?!)
# Reference: https://stackoverflow.com/questions/10291997/how-can-i-do-assignments-in-a-list-comprehension

In [68]:
# Get the index of loaded image
imgIDs = [[i.split('_')[0] for i in j.attrs['long_name']] for j in xdas]
print(imgIDs)
# Get years based on index
t = [[pair[i] for i in j] for j in imgIDs]
print(t)

# List Comprehension to create List of Lists
# Reference: https://blog.finxter.com/list-comprehension-python-list-of-lists/

[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35'], ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35'], ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35'], ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']]
[[1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,

In [71]:
# Replace "index" coordinates with "year" coordinates
for i in range(len(t)):
  xdss[i].coords['time'] = np.array(t[i])
  
xdss

[<xarray.Dataset>
 Dimensions:      (time: 36, x: 11008, y: 4484)
 Coordinates:
   * y            (y) float64 34.08 34.08 34.08 34.08 ... 35.29 35.29 35.29 35.29
   * time         (time) int64 1984 1985 1986 1987 1988 ... 2016 2017 2018 2019
   * x            (x) float64 132.5 132.5 132.5 132.5 ... 135.5 135.5 135.5 135.5
     spatial_ref  int64 0
 Data variables:
     watermask    (time, y, x) uint8 ...
 Attributes:
     parcelID:  151, <xarray.Dataset>
 Dimensions:      (time: 36, x: 527, y: 4484)
 Coordinates:
   * y            (y) float64 34.08 34.08 34.08 34.08 ... 35.29 35.29 35.29 35.29
   * time         (time) int64 1984 1985 1986 1987 1988 ... 2016 2017 2018 2019
   * x            (x) float64 135.5 135.5 135.5 135.5 ... 135.6 135.6 135.6 135.6
     spatial_ref  int64 0
 Data variables:
     watermask    (time, y, x) uint8 ...
 Attributes:
     parcelID:  151, <xarray.Dataset>
 Dimensions:      (time: 34, x: 11264, y: 11264)
 Coordinates:
   * y            (y) float64 21.59 21.

## Subset Ponds to Parcel Level
*After finishing the calculation of one parcel, delete that from RAM before subsetting the next parcel.*

In [ ]:
ponds_sub = ponds_chn[ponds_chn.geometry.within(parcels[parcels['id']==parcelID[11]].geometry.unary_union)]
ponds_sub

,id,index_right,GID_0,NAME_0,geometry
831974,885556,2,CHN,China,"MULTIPOLYGON (((115.85321 23.41926, 115.85321 ..."
831975,885557,2,CHN,China,"MULTIPOLYGON (((115.85482 23.42285, 115.85482 ..."
831976,885558,2,CHN,China,"MULTIPOLYGON (((115.85608 23.41037, 115.85653 ..."
831977,885559,2,CHN,China,"MULTIPOLYGON (((115.85707 23.41225, 115.85734 ..."
831978,885560,2,CHN,China,"MULTIPOLYGON (((115.85815 23.41046, 115.85815 ..."
...,...,...,...,...,...
885710,941218,2,CHN,China,"MULTIPOLYGON (((114.68297 22.92878, 114.68297 ..."
885712,941220,2,CHN,China,"MULTIPOLYGON (((114.68701 22.91782, 114.68710 ..."
885714,941222,2,CHN,China,"MULTIPOLYGON (((114.68836 22.91414, 114.68863 ..."
885748,941259,2,CHN,China,"MULTIPOLYGON (((114.74747 22.73456, 114.74801 ..."


## Test: Zonal Statistics using **`xarray-_spatial`**


In [ ]:
#import dask
#from xrspatial import zonal_stats
import warnings
warnings.filterwarnings("ignore")

***For `zonal_stats` to work, make sure that*** <br>
* `array` is a 2D array;

* `transform` is written in this form:
1. top left easting or X coordinate
2. West-East pixel resolution
3. rotation (typically 0 if image is "north up")
4. top left northing or Y coordinate
5. rotation, (typically 0 if image is "north up")
6. North-South pixel resolution (typically -1 * W-E resolution)



In [ ]:
xda_test = xds.sel(time=2019).to_array()
xda_test

<xarray.DataArray (variable: 1, y: 11264, x: 11264)>
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)
Coordinates:
  * y            (y) float64 21.59 21.59 21.59 21.59 ... 24.62 24.62 24.62 24.62
    time         int64 2019
  * x            (x) float64 114.3 114.3 114.3 114.3 ... 117.3 117.3 117.3 117.3
    spatial_ref  int64 0
  * variable     (variable) <U9 'watermask'

In [ ]:
# Convert DataArray to numpy array
# https://stackoverflow.com/questions/55132416/converting-dataarray-into-numpy-array
array = xda_test.values[0]
array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
# Get "Affine transform"
# https://github.com/perrygeo/python-rasterstats/blob/master/docs/notebooks/Integrating%20with%20GeoPandas%20and%20Numpy.ipynb

transform = xda_test.rio.transform()
we_pr, rotation, ns_pr = transform[0], transform[1], transform[4]

x, y = xds.x.min().values, xds.y.max().values

transform = [x, we_pr, rotation, y, rotation, ns_pr]

transform

[array(114.25964051),
 0.00026949458523585647,
 0.0,
 array(24.62062107),
 0.0,
 -0.00026949458523585647]

In [ ]:
zonstats = zonal_stats(ponds_test4, array, transform=transform, categorical=True, all_touched=True) # 
zonstats

/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/usr/local/lib/python3.7/dist-

[{0: 4},
 {0: 32, 1: 2},
 {0: 1},
 {1: 4},
 {0: 9},
 {1: 6},
 {0: 33},
 {0: 16},
 {0: 10},
 {0: 29, 1: 1},
 {0: 2},
 {1: 6},
 {0: 39},
 {0: 4},
 {0: 1},
 {0: 11},
 {0: 8},
 {1: 8},
 {0: 15},
 {0: 7}]

* **Add Zonal Statistics to Geodataframe**

In [ ]:
df_stats = pd.DataFrame(zonstats)

# Convert data type of columns
# https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
df_stats = df_stats.convert_dtypes()

# Rename columns
df_stats.rename(columns={0:'c0_2019', 1:'c1_2019'}, inplace=True)

# Add 'pondID' as new column
df_stats = df_stats.assign(pondID = np.array(ponds_test4['pondID']))
df_stats

,c0_2019,c1_2019,pondID
0,4,<NA>,894759
1,32,2,894761
2,1,<NA>,894762
3,<NA>,4,894763
4,9,<NA>,894764
5,<NA>,6,894765
6,33,<NA>,894766
7,16,<NA>,894767
8,10,<NA>,894769
9,29,1,894770


In [ ]:
# Add Zonal Stats to GDF
ponds_zonstats = ponds_test4.join(df_stats.set_index('pondID'), on='pondID')
ponds_zonstats

,pondID,index_right,GID_0,NAME_0,geometry,c0_2019,c1_2019
841034,894759,2,CHN,China,"MULTIPOLYGON (((116.28314 23.00002, 116.28332 ...",4,<NA>
841035,894761,2,CHN,China,"MULTIPOLYGON (((116.28323 22.98259, 116.28341 ...",32,2
841036,894762,2,CHN,China,"MULTIPOLYGON (((116.28332 23.21417, 116.28332 ...",1,<NA>
841037,894763,2,CHN,China,"MULTIPOLYGON (((116.28341 23.36087, 116.28350 ...",<NA>,4
841038,894764,2,CHN,China,"MULTIPOLYGON (((116.28323 22.97603, 116.28359 ...",9,<NA>
841039,894765,2,CHN,China,"MULTIPOLYGON (((116.28332 23.38216, 116.28359 ...",<NA>,6
841040,894766,2,CHN,China,"MULTIPOLYGON (((116.28305 22.96390, 116.28368 ...",33,<NA>
841041,894767,2,CHN,China,"MULTIPOLYGON (((116.28359 22.97953, 116.28368 ...",16,<NA>
841042,894769,2,CHN,China,"MULTIPOLYGON (((116.28368 22.96157, 116.28377 ...",10,<NA>
841043,894770,2,CHN,China,"MULTIPOLYGON (((116.28278 22.96274, 116.28377 ...",29,1
